# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict
import functools

In [36]:
DEBAG_MODE = True
def mprint(*args, **kwargs):
    if DEBAG_MODE:
        print(*args, **kwargs)

In [10]:
class LruCache(object):
    def __init__(self, max_count):
        self.dict = OrderedDict()
        self.max_count = max_count

    def __getitem__(self, key):
        mprint('Hallo Get!')
        return self.dict[key]
    
    def __setitem__(self, key, value):
        mprint('Hallo Set!', key)
        self.dict[key] = value
        while len(self.dict) > self.max_count:
            mprint('Remove from cache: ', list(self.dict.keys())[0])
            self.dict.popitem(last=False)
        
    def cached(self, key):
        try:
            value = self.dict[key]
            return True, value
        except:
            return False, _
        
        
    pass

#### Реализуйте декоратор

In [11]:
def cached(max_count):
    cache = LruCache(max_count)
    mprint('Cache created! Cache size: {0}'.format(max_count))
    def deprecated(func):
        mprint('Func deprecated!')
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            mprint('Func {0} wrapped! Arg: {1}'.format(func.__name__, args[0]))
            cres, res = cache.cached((args, *kwargs))
            if cres:
                mprint('Cached!', res)
            else:
                res = func(*args, **kwargs)
                cache[(args, *kwargs)] = res
            return res
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [12]:
@cached(10)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(5)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [13]:
print(fact(15))
print(len(fact.__closure__[0].cell_contents.dict), fact.__closure__[0].cell_contents.dict)

print(fact(12))

print(fact1(8))

print(fact(10))

print(len(fact.__closure__[0].cell_contents.dict), fact.__closure__[0].cell_contents.dict)

print(len(fact1.__closure__[0].cell_contents.dict), fact1.__closure__[0].cell_contents.dict)

1307674368000
10 OrderedDict([(((6,),), 720), (((7,),), 5040), (((8,),), 40320), (((9,),), 362880), (((10,),), 3628800), (((11,),), 39916800), (((12,),), 479001600), (((13,),), 6227020800), (((14,),), 87178291200), (((15,),), 1307674368000)])
479001600
40320
3628800
10 OrderedDict([(((6,),), 720), (((7,),), 5040), (((8,),), 40320), (((9,),), 362880), (((10,),), 3628800), (((11,),), 39916800), (((12,),), 479001600), (((13,),), 6227020800), (((14,),), 87178291200), (((15,),), 1307674368000)])
5 OrderedDict([(((4,),), 24), (((5,),), 120), (((6,),), 720), (((7,),), 5040), (((8,),), 40320)])


#### Сравните свою реализацию с lru_cache из functools

In [14]:
from functools import lru_cache

@lru_cache(8)
def fact2(n):
    if n < 2:
        return 1
    return fact2(n-1) * n

print(fact2(21))
print(dir(fact2.cache_info))
print(fact2.cache_info)

fact2.cache_info()

51090942171709440000
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']
<built-in method cache_info of functools._lru_cache_wrapper object at 0x104b89390>


CacheInfo(hits=0, misses=21, maxsize=8, currsize=8)

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [16]:
#В LruCache добавилась функция cached, которая возвращает значение,
#которое уже ранее возвращала функция при тех же переданных аргументах

#UPD: нужно в OrderedList передавать не только argv и kwargs, но и переменные, которые функция получает извне (глобольные, например)

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [45]:
def checked(*types):
    mprint('Checker created!')
    def deprecated(func):
        mprint('Func deprecated!')
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            mprint('Func {0} wrapped!'.format(func.__name__))
            mprint(len(types), len(args))
            if len(types) != len(args):
                raise TypeError
            for key_idx in range(len(args)):
                mprint(types[key_idx], args[key_idx], type(args[key_idx]))
                if not types[key_idx] is type(args[key_idx]):
                    raise TypeError    
            return func(*args, **kwargs)
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [46]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    return a + '|' + str(b) + '|' + 'O'.join(c)

Checker created!
Func deprecated!


In [47]:
strange_func('Ja', 5, ['1', '2', '3'])

Func strange_func wrapped!
3 3
<class 'str'> Ja <class 'str'>
<class 'int'> 5 <class 'int'>
<class 'list'> ['1', '2', '3'] <class 'list'>


'Ja|5|1O2O3'

In [48]:
strange_func('Ja', '5', ['1', '2', '3'])

Func strange_func wrapped!
3 3
<class 'str'> Ja <class 'str'>
<class 'int'> 5 <class 'str'>


TypeError: 

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [ ]:
<your code here>